In [6]:
# importar librerias
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import ast # tranforma a lista

import plotly.express as px

# 1. Extract and check the data

In [7]:
# read the data
df_raw = pd.read_csv('gsearch_jobs.csv')


* to delete: ['Unnamed: 0','index', 'job_id']
* others columns: ['company_name', 'location', 'via', 'description', 'extensions']
* to use:  ['description_tokens', 'posted_at', 'date_time']
* to check: ['title', 'search_location', 'location', 'posted_at', 'search_term']

In [8]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61953 entries, 0 to 61952
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           61953 non-null  int64  
 1   index                61953 non-null  int64  
 2   title                61953 non-null  object 
 3   company_name         61953 non-null  object 
 4   location             61916 non-null  object 
 5   via                  61944 non-null  object 
 6   description          61953 non-null  object 
 7   extensions           61953 non-null  object 
 8   job_id               61953 non-null  object 
 9   thumbnail            38194 non-null  object 
 10  posted_at            61763 non-null  object 
 11  schedule_type        61707 non-null  object 
 12  work_from_home       27980 non-null  object 
 13  salary               10088 non-null  object 
 14  search_term          61953 non-null  object 
 15  date_time            61953 non-null 

* Check others columns to decide: 
    * Hace cuanto tiempo fues posteado no me interera **posted_at**
    * Me interesa la fecha del post **date_time**
    * **description_tokens** algunos vacios los eliminare
    * **location** no usare porque todo es USA
    * title los titulos de rol pueden variar pero el rol puede ser uno solo, no me sirve **title**
    * solo busqueda de DA, no me sirve **search_term**

In [9]:
df_raw[['title', 'search_location','location', 'posted_at','search_term', 'description_tokens', 'date_time']]

,title,search_location,location,posted_at,search_term,description_tokens,date_time
0,Data Analyst,United States,Anywhere,15 hours ago,data analyst,"['tableau', 'r', 'python', 'sql']",2023-08-04 03:00:13.797776
1,Data Analyst,United States,United States,12 hours ago,data analyst,[],2023-08-04 03:00:13.797776
2,Aeronautical Data Analyst,United States,"Olathe, KS",18 hours ago,data analyst,['sql'],2023-08-04 03:00:13.797776
3,Data Analyst - Consumer Goods - Contract to Hire,United States,Anywhere,12 hours ago,data analyst,"['powerpoint', 'excel', 'power_bi']",2023-08-04 03:00:13.797776
4,Data Analyst | Workforce Management,United States,United States,7 hours ago,data analyst,"['powerpoint', 'excel', 'outlook', 'word']",2023-08-04 03:00:13.797776
...,...,...,...,...,...,...,...
61948,Marketing Data & BI Analyst II,United States,"Houstonia, MO",23 hours ago,data analyst,"['power_bi', 'tableau', 'excel', 'snowflake', ...",2022-11-04 03:40:23.706734
61949,Lead-Data Analyst,United States,"Marshfield, MO",23 hours ago,data analyst,[],2022-11-24 04:00:08.710801
61950,Lead-Data Analyst,United States,"High Point, MO",23 hours ago,data analyst,[],2022-12-07 04:00:12.563831
61951,Lead-Data Analyst,United States,"Calhoun, MO",23 hours ago,data analyst,[],2022-12-08 04:00:15.975728


In [10]:
for i in ['title', 'search_location', 'location', 'search_term']:
    print(f'Datos únicos de {i}:', df_raw[i].unique())                          

Datos únicos de title: ['Data Analyst' 'Aeronautical Data Analyst'
 'Data Analyst - Consumer Goods - Contract to Hire' ...
 'Data Analyst - Business Intelligence' 'COOP - Senior Data Analyst'
 'Lead FP&A Analyst- Remote, US']
Datos únicos de search_location: ['United States']
Datos únicos de location: [' Anywhere ' '  United States   ' '  Olathe, KS   ' ... 'Eudora, MO'
 'Marionville, MO' 'Salem, MO']
Datos únicos de search_term: ['data analyst']


# 2. Define el dataset final

Acomodar el dataset para trabajarlo
1. la columna description_tokens no esta como lista, hay que transformarlo
2. elimino las listas vacias, las que no tienen skills
3. separo los skills de la lista
4. tranformar la columna date_time a fechas y mes
5. ordeno los skills segun fecha de aparacion

In [11]:
df = (df_raw[['description_tokens', 'date_time']]
    .assign(skills = lambda df: df['description_tokens'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else [])) # tranformacion a lista
    .loc[lambda df: df['skills'].str.len() > 0] #elimino tokens vacios y como esta con string esta contango 2 [ y ]
    .explode('skills')
    .assign(date_time = lambda df: pd.to_datetime(df['date_time']).dt.to_period('M')) # tranformo y obengo solo meses
    .sort_values(['date_time'])# ordenar ates de agrupar
)[['date_time', 'skills']]


df.head(5)

,date_time,skills
58621,2022-11,sas
58621,2022-11,excel
58621,2022-11,tableau
58620,2022-11,vba
58620,2022-11,microstrategy


In [12]:
(df.groupby(['date_time'])
              .size()
              .reset_index(name='count')
              .sort_values('date_time')
           )

,date_time,count
0,2022-11,7796
1,2022-12,10842
2,2023-01,12118
3,2023-02,9214
4,2023-03,8876
5,2023-04,7816
6,2023-05,7300
7,2023-06,7720
8,2023-07,8173
9,2023-08,10870


* Contamos la frecuencia mensual de cada skill
* Seleccionamos un grupo de skill top para eliminar el ruido
* df final a trabajar ya filtrado con el top

In [13]:
# cuenta mensual de skill
df_count = (df.groupby(['date_time', 'skills'])
              .size()
              .reset_index(name='count') 
           )
df_count.head()

,date_time,skills,count
0,2022-11,airflow,19
1,2022-11,alteryx,45
2,2022-11,asp.net,3
3,2022-11,assembly,4
4,2022-11,atlassian,6


In [14]:
N_TOP_SKILLS = 50
top_skills = (df_count
              .groupby('skills')['count']
              .sum()
              .nlargest(N_TOP_SKILLS).index # reduccion de ruido (top skills)
             )
top_skills

Index(['sql', 'excel', 'python', 'power_bi', 'tableau', 'r', 'sas',
       'powerpoint', 'word', 'azure', 'snowflake', 'aws', 'spark', 'looker',
       'go', 'spreadsheet', 'sap', 'scala', 'spss', 'jira', 'java', 'c',
       'alteryx', 'outlook', 'hadoop', 'nosql', 'sharepoint', 'vba',
       'javascript', 'gcp', 'bigquery', 'redshift', 'qlik', 'ssrs', 'ssis',
       'pandas', 'microstrategy', 'git', 'mysql', 'dax', 'visio', 'cognos',
       'matplotlib', 'tensorflow', 'pyspark', 'airflow', 'numpy', 'linux',
       'c++', 'pl/sql'],
      dtype='object', name='skills')

In [15]:
df_final = df_count[df_count['skills'].isin(top_skills)]
df_final.head()

,date_time,skills,count
0,2022-11,airflow,19
1,2022-11,alteryx,45
6,2022-11,aws,133
7,2022-11,azure,124
9,2022-11,bigquery,23


# 3. Definir metricas estrategicas

1. **crecimiento anual**: Crecimiento Interanual
2. **ratio sharpie 6 meses**: crecimiento al rededor del ruido generado.
3. **frecuencia absoluta**: nro de recurrencia del puesto.

In [16]:
df_final = (df_final
    .assign(crec_mensual = lambda d: d.groupby('skills')['count']
                                   .pct_change(periods=1) # cambio porcentual mensual 
                                   .fillna(0))
    .assign(crec_ewm_6m = lambda d: d.groupby('skills')['crec_mensual']
                                    .ewm(span=6).mean()
                                    .reset_index(level=0, drop=True)) # media movil de los 6 meses de crecimiento 
    .assign(volatil_6m = lambda d: d.groupby('skills')['crec_mensual']
                                    .ewm(span=6).std()
                                    .reset_index(level=0, drop=True))
    .assign(ratio_sharpe = lambda d: d.crec_ewm_6m / d.volatil_6m)
    .assign(ratio_sharpe = lambda d: d['ratio_sharpe']
                                     .replace([np.inf, -np.inf], np.nan)
                                     .fillna(0))
    .assign(crec_YoY = lambda d: d.groupby('skills')['count']
                                  .pct_change(periods=12))
    .assign(crec_ultimos_6m = lambda d: d.groupby('skills')['count']
                                     .pct_change(periods=6)
                                     .fillna(0) # Se rellena con 0 si no hay data suficiente o previa
    )# crecimiento anual
) 

df_final.head()

,date_time,skills,count,crec_mensual,crec_ewm_6m,volatil_6m,ratio_sharpe,crec_YoY,crec_ultimos_6m
0,2022-11,airflow,19,0.0,0.0,NaN,0.0,NaN,0.0
1,2022-11,alteryx,45,0.0,0.0,NaN,0.0,NaN,0.0
6,2022-11,aws,133,0.0,0.0,NaN,0.0,NaN,0.0
7,2022-11,azure,124,0.0,0.0,NaN,0.0,NaN,0.0
9,2022-11,bigquery,23,0.0,0.0,NaN,0.0,NaN,0.0


# 4. Analisis de las metricas creadas

In [17]:
# metricas del ultimo mes
df_metrics = (df_final
     .groupby('skills')
     .last()
     .reset_index()
     .dropna(subset=['crec_YoY']) # eliminar los que tienes menos de 1 year
     .assign(crec_YoY = lambda d: d['crec_YoY'].mul(100)) # Convertir a porcentaje (%)
     .assign(crec_ultimos_6m = lambda d: d['crec_ultimos_6m'].mul(100)) # Convertir a %
)[['skills', 'count', 'crec_YoY', 'crec_ultimos_6m', 'ratio_sharpe']]

df_metrics.head()

,skills,count,crec_YoY,crec_ultimos_6m,ratio_sharpe
0,airflow,10,-72.972973,-28.571429,0.133498
1,alteryx,24,-47.826087,-4.000000,0.186998
2,aws,40,-75.903614,-34.426230,-0.127889
3,azure,52,-76.255708,30.000000,0.048886
4,bigquery,3,-94.000000,-84.210526,-0.239278


* Frecuencia Absoluta. Es el número total de menciones del skill en el período de tiempo más reciente (tu última observación, Abril 2025).
* Crecimiento Interanual (Year-over-Year). El cambio porcentual en la demanda del skill al comparar el último mes (Abril 2025) con el mismo mes del año anterior (Abril 2024). Indica si el skill ha resistido o ha sido víctima de los recortes del mercado en el último año.
* Impulso Reciente (Growth Momentum). El cambio porcentual en la demanda del skill al comparar el último Mide la Recuperación o Impulso Actual. Es la métrica más predictiva, indicando si el skill se está reactivando ahora.
* Ratio de Sharpe (Estabilidad/Riesgo). Mide el rendimiento (crecimiento promedio reciente, EWMA) ajustado por su volatilidad (desviación estándar, EWMA).Calidad del Crecimiento. Un valor alto indica que el skill tiene una demanda estable y predecible (bajo riesgo), mientras que un valor bajo o negativo indica demanda errática y volátil (alto riesgo)

# 5. Definimos nuestro grupo de skills


In [18]:
# Calcular los umbrales (medianas)
umbral_crec_6m = 0
#umbral_yoy = df_metrics['crec_YoY'].median()
umbral_sharpe = df_metrics['ratio_sharpe'].median()

# define segun el cuadrante donde se ubica
def asignar_cuadrante(row):
    # Crecimiento (Eje X)
    crecimiento_alto = row['crec_ultimos_6m'] >= umbral_crec_6m
    
    # Estabilidad/Sharpe (Eje Y)
    estabilidad_alta = row['ratio_sharpe'] >= umbral_sharpe
    
    if crecimiento_alto and estabilidad_alta:
        return '1. Inversión de Crecimiento Sólido'
    elif not crecimiento_alto and estabilidad_alta:
        return '2. Valor de Portafolio Estable'
    elif crecimiento_alto and not estabilidad_alta:
        return '3. Activos de Alto Beta / Volatilidad Extrema'
    else:
        return '4. Activos en Liquidación'

# clasificacion del skill
df_metrics['clasificacion'] = df_metrics.apply(asignar_cuadrante, axis=1)

In [19]:
df_metrics.head()

,skills,count,crec_YoY,crec_ultimos_6m,ratio_sharpe,clasificacion
0,airflow,10,-72.972973,-28.571429,0.133498,2. Valor de Portafolio Estable
1,alteryx,24,-47.826087,-4.000000,0.186998,2. Valor de Portafolio Estable
2,aws,40,-75.903614,-34.426230,-0.127889,4. Activos en Liquidación
3,azure,52,-76.255708,30.000000,0.048886,3. Activos de Alto Beta / Volatilidad Extrema
4,bigquery,3,-94.000000,-84.210526,-0.239278,4. Activos en Liquidación


In [20]:
df_metrics.to_csv('df_metrics.csv', index=False, sep=';')

## 5.1 Estrella (Alto Crecimiento Estable)

In [21]:
df_metrics.query("clasificacion == '1. Inversión de Crecimiento Sólido'")

,skills,count,crec_YoY,crec_ultimos_6m,ratio_sharpe,clasificacion
6,c++,21,-4.545455,0.000000,0.302133,1. Inversión de Crecimiento Sólido
11,git,22,-50.000000,15.789474,0.231495,1. Inversión de Crecimiento Sólido
12,go,49,-65.248227,88.461538,0.215029,1. Inversión de Crecimiento Sólido
13,hadoop,17,-63.829787,0.000000,0.187698,1. Inversión de Crecimiento Sólido
14,java,40,-35.483871,81.818182,0.128905,1. Inversión de Crecimiento Sólido
15,javascript,34,-40.350877,25.925926,0.186824,1. Inversión de Crecimiento Sólido
16,jira,23,-52.083333,4.545455,0.206601,1. Inversión de Crecimiento Sólido
21,mysql,14,-67.441860,75.000000,0.221985,1. Inversión de Crecimiento Sólido
23,numpy,13,-60.606061,85.714286,0.180801,1. Inversión de Crecimiento Sólido
25,pandas,18,-50.000000,50.000000,0.175946,1. Inversión de Crecimiento Sólido


## 5.2 Maduro Estable (Bajo Riesgo)

In [22]:
df_metrics.query("clasificacion == '2. Valor de Portafolio Estable'")

,skills,count,crec_YoY,crec_ultimos_6m,ratio_sharpe,clasificacion
0,airflow,10,-72.972973,-28.571429,0.133498,2. Valor de Portafolio Estable
1,alteryx,24,-47.826087,-4.000000,0.186998,2. Valor de Portafolio Estable
7,cognos,22,-15.384615,-4.347826,0.283503,2. Valor de Portafolio Estable
8,dax,19,-48.648649,-17.391304,0.329826,2. Valor de Portafolio Estable
19,matplotlib,2,-94.444444,-71.428571,0.100152,2. Valor de Portafolio Estable
20,microstrategy,4,-71.428571,-20.000000,0.290253,2. Valor de Portafolio Estable
24,outlook,19,-52.500000,-32.142857,0.174600,2. Valor de Portafolio Estable
26,pl/sql,8,-68.000000,-50.000000,0.176007,2. Valor de Portafolio Estable
31,qlik,8,-79.487179,-11.111111,0.148590,2. Valor de Portafolio Estable
48,visio,7,-66.666667,-30.000000,0.172796,2. Valor de Portafolio Estable


## 5.3 Apuesta Volátil (Alto Riesgo)

In [23]:
df_metrics.query("clasificacion == '3. Activos de Alto Beta / Volatilidad Extrema'")

,skills,count,crec_YoY,crec_ultimos_6m,ratio_sharpe,clasificacion
3,azure,52,-76.255708,30.000000,0.048886,3. Activos de Alto Beta / Volatilidad Extrema
9,excel,305,-53.927492,34.955752,0.007194,3. Activos de Alto Beta / Volatilidad Extrema
17,linux,9,-75.000000,12.500000,-0.172792,3. Activos de Alto Beta / Volatilidad Extrema
27,power_bi,252,-63.688761,19.431280,-0.091255,3. Activos de Alto Beta / Volatilidad Extrema
30,python,278,-65.379826,14.403292,-0.122455,3. Activos de Alto Beta / Volatilidad Extrema
32,r,148,-66.741573,2.777778,-0.146877,3. Activos de Alto Beta / Volatilidad Extrema
35,sas,79,-48.366013,9.722222,-0.022796,3. Activos de Alto Beta / Volatilidad Extrema
38,snowflake,54,-57.812500,22.727273,-0.013055,3. Activos de Alto Beta / Volatilidad Extrema
42,sql,492,-57.622739,37.815126,-0.013295,3. Activos de Alto Beta / Volatilidad Extrema
44,ssrs,15,-53.125000,15.384615,0.069958,3. Activos de Alto Beta / Volatilidad Extrema


## 5.4 En Declive (Bajo Crecimiento Volátil)'

In [24]:
# skills declive
df_metrics.query("clasificacion == '4. Activos en Liquidación'")

,skills,count,crec_YoY,crec_ultimos_6m,ratio_sharpe,clasificacion
2,aws,40,-75.903614,-34.426230,-0.127889,4. Activos en Liquidación
4,bigquery,3,-94.000000,-84.210526,-0.239278,4. Activos en Liquidación
5,c,15,-76.190476,-31.818182,-0.046565,4. Activos en Liquidación
10,gcp,12,-78.181818,-53.846154,0.033149,4. Activos en Liquidación
18,looker,32,-67.676768,-3.030303,-0.220881,4. Activos en Liquidación
22,nosql,9,-82.692308,-18.181818,-0.060261,4. Activos en Liquidación
28,powerpoint,60,-47.826087,-4.761905,0.022729,4. Activos en Liquidación
29,pyspark,7,-81.081081,-30.000000,0.037888,4. Activos en Liquidación
36,scala,11,-87.912088,-68.571429,0.064629,4. Activos en Liquidación
37,sharepoint,13,-71.739130,-31.578947,-0.245932,4. Activos en Liquidación


# 6. Vista grafica de los grupos

In [25]:
# Renombrar para que los nombres en el gráfico sean claros
df_plot = df_metrics.rename(columns={
    'count': 'Frecuencia Absoluta',
    'crec_ultimos_6m': 'Crecimiento Últimos 6m (%)',
    #'crec_YoY': 'Crecimiento Interanual (%)',
    'ratio_sharpe': 'Ratio Sharpe (Estabilidad)'
})
df_plot.head()

fig = px.scatter(
    df_plot,
    x='Crecimiento Últimos 6m (%)',
    y='Ratio Sharpe (Estabilidad)',
    size='Frecuencia Absoluta',  # El tamaño de la burbuja
    color='clasificacion',             # El color se define por el cuadrante
    hover_name='skills',         # Mostrar el nombre del skill al pasar el ratón
    log_x=False,                 # Desactivar escala logarítmica si no es necesaria
    size_max=50,                 # Tamaño máximo de burbuja para evitar que sean muy grandes
    title='Análisis de Evolución de Skills: Crecimiento vs. Estabilidad (Ratio Sharpe)',
    category_orders={
        'clasificacion': [
            '1. Inversión de Crecimiento Sólido',
            '2. Valor de Portafolio Estable', 
            '3. Activos de Alto Beta / Volatilidad Extrema',
            '4. Activos en Liquidación'
        ]
    }
)

fig.add_vline(x=umbral_crec_6m, line_width=1, line_dash="dash", line_color="gray")
fig.add_hline(y=umbral_sharpe, line_width=1, line_dash="dash", line_color="gray")

# Etiquetas claras para los ejes
fig.update_layout(
    xaxis_title="Crecimiento Últimos 6m (%)",
    yaxis_title="Ratio de Sharpe (Estabilidad / Bajo Riesgo)"
)

fig.show()